In [93]:
import pandas as pd
noise_campaign_data = pd.read_csv("C:\\Users\\hanna\\Desktop\\data\\grafana_stopped.csv", delimiter=',',  parse_dates=True)
noise_campaign_data

,Time,10min_sample.wind_direction,10min_sample.wind_speed,10min_sample.speed_bin,10min_sample.direction_bin
0,2020-12-26 18:43:12,196.0,13.20,13.0,210
1,2020-12-26 18:53:12,199.0,14.50,NaN,210
2,2020-12-26 19:03:12,197.0,12.10,12.0,210
3,2020-12-26 19:13:12,191.0,12.00,12.0,180
4,2020-12-26 19:22:43,190.0,11.90,12.0,180
...,...,...,...,...,...
1066,2021-01-13 06:45:45,16.0,6.33,6.0,30
1067,2021-01-13 06:55:45,11.7,6.22,6.0,0
1068,2021-01-13 07:15:45,12.9,6.11,6.0,0
1069,2021-01-13 08:05:16,16.5,6.93,7.0,30


In [94]:
grouped = noise_campaign_data.groupby(['10min_sample.speed_bin', '10min_sample.direction_bin']).size()

grouped = noise_campaign_data.groupby(['10min_sample.speed_bin', '10min_sample.direction_bin'], as_index=False).count().reset_index()[['10min_sample.speed_bin', '10min_sample.direction_bin', "Time"]]

grouped

,10min_sample.speed_bin,10min_sample.direction_bin,Time
0,3.0,0,4
1,3.0,30,8
2,3.0,60,14
3,3.0,90,10
4,3.0,120,12
...,...,...,...
78,13.0,210,4
79,13.0,330,3
80,14.0,150,2
81,14.0,180,2


In [95]:
filtered = grouped[grouped['Time'] > 2].copy()
filtered['diff'] = filtered['Time'] - 2
wrong_stops_counts = filtered['diff'].sum()

In [96]:
wrong_stops_counts

160

In [97]:
total_stops = grouped['Time'].sum()
total_stops

319

In [98]:
wrong_stopped_hours = (wrong_stops_counts * 10) / 60
total_stopped_hours = (total_stops * 10) / 60

In [99]:
wrong_stopped_hours

26.666666666666668

In [100]:
total_stopped_hours

53.166666666666664

In [101]:
wrong_percentage = wrong_stopped_hours / total_stopped_hours * 100
wrong_percentage

50.15673981191223

In [102]:
import datetime as dt

minimum = dt.datetime.strptime(
        noise_campaign_data['Time'].min(), "%Y-%m-%d %H:%M:%S"
    )
minimum

datetime.datetime(2020, 12, 26, 18, 43, 12)

In [103]:
maximum = dt.datetime.strptime(
        noise_campaign_data['Time'].max(), "%Y-%m-%d %H:%M:%S"
    )
maximum

datetime.datetime(2021, 1, 13, 8, 15, 16)

In [104]:
scada_data = pd.read_csv("C:\\Users\\hanna\\Documents\\GitHub\\dissertation\\single_turbine_varda.csv", delimiter=',')
scada_data['timestamp'] =  pd.to_datetime(scada_data['timestamp'], format='%Y-%m-%d %H:%M:%S')

In [105]:
filtered_scada = scada_data[(scada_data['timestamp'] <= maximum)][(scada_data['timestamp'] >= minimum)]

<ipython-input-105-3cfbb2c16d08>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_scada = scada_data[(scada_data['timestamp'] <= maximum)][(scada_data['timestamp'] >= minimum)]


In [106]:
filtered_scada.index = filtered_scada['timestamp']

In [107]:
resampled = filtered_scada.resample('10min', origin='start').mean()

In [108]:
resampled

,Unnamed: 0,wind_speed,wind_direction
timestamp,,,
2020-12-26 18:43:12.579,5771.5,14.160,198.600000
2020-12-26 18:53:12.579,5911.5,12.060,195.914999
2020-12-26 19:03:12.579,6051.5,11.615,191.405001
2020-12-26 19:13:12.579,6191.5,11.805,189.710001
2020-12-26 19:23:12.579,6331.5,10.810,192.864999
...,...,...,...
2021-01-13 07:33:12.579,359159.5,5.795,18.980000
2021-01-13 07:43:12.579,359299.5,6.690,19.665000
2021-01-13 07:53:12.579,359439.5,7.110,15.950000
